In [1]:
from pyspark import SparkContext
from scipy import sparse as sm
from sklearn.preprocessing import normalize
import numpy as np
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import spearmanr
from scipy.stats import pearsonr as pears
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
import time
from sparsesvd import sparsesvd
import math as mt
sc = SparkContext.getOrCreate()

In [2]:
train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1])))
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(','))

test_users=test_clean_data.map( lambda x: int(x[0])).collect()


grouped_rates = train_clean_data.filter(lambda x: x[0] in test_users).map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
grouped_rates_dic = dict(grouped_rates)


item_ratings = train_clean_data.map(lambda x: (x[0], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))
user_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1]))).collect()
user_ratings_mean_dic=dict(user_ratings_mean)


item_ratings_forTop = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
shrinkage_factor = 5
item_ratings_mean = item_ratings_forTop.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()


#matrix interactions with global effect creation
users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
UxI_basic= sm.csr_matrix((ratings, (users, items)))

#remove global effect
global_mean = train_clean_data.map(lambda x: x[2]).mean()

trainr2 = train_clean_data.map(lambda x: (x[0], x[1], x[2]-global_mean))
temp = trainr2.map(lambda x: (x[1],(x[2],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
i_Bias = temp.collectAsMap()

trainr3 = trainr2.map(lambda x: (x[0],x[1], x[2] - i_Bias.get(x[1],0)))
temp2 = trainr3.map(lambda x: (x[0],(x[2],1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])
u_Bias = temp2.collectAsMap()

train_clean_data = train_clean_data.map(lambda x: (x[0], x[1], x[2] - i_Bias.get(x[1],0) - u_Bias.get(x[0],0)))
train_clean_data.take(5)

#matrix interactions without global effect creation
users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
UxI_global1= sm.csr_matrix((ratings, (users, items)))

#matrix icm creations
items_for_features= icm_clean_data.map(lambda x:x[0]).collect()
features = icm_clean_data.map(lambda x:x[1]).collect()
items_for_features.append(37142)
features.append(0)
ones=[1]*len(items_for_features)
IxF= sm.csr_matrix((ones, (items_for_features, features)))

In [3]:
n_users,n_items=UxI_basic.shape
n_features=IxF.shape[1]

SVD

In [4]:
'''matrix factorization -  TUNE PARAMETER HERE'''
K = 870
%time U, S, Vt = sparsesvd(UxI_basic.tocsc(), K)
S=np.sqrt(S.data)
S=np.diag(S)

CPU times: user 1min 40s, sys: 148 ms, total: 1min 40s
Wall time: 1min 40s


In [5]:
'''calc predictions'''
%time UxI_pred_SVD=sm.csr_matrix(U.T.dot(S).dot(Vt))

CPU times: user 48.1 s, sys: 6.09 s, total: 54.2 s
Wall time: 27.7 s


Content Based

In [6]:
IDF=[0]*n_features
for i in tqdm(range(n_features)):
    IDF[i]=np.log10(n_items/len(IxF.getcol(i).nonzero()[1]))
%time IxF=normalize(IxF,axis=1) #apply TF
%time IxF_idf=IxF.multiply(IDF) #apply IDF
%time UxF=UxI_global1.dot(IxF_idf) #calc users profiles


CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.4 ms
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.36 ms
CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 58.1 ms


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [7]:
'''calc predictions'''
%time UxI_pred_CB=UxF.dot(IxF.T)
del IxF

CPU times: user 8.22 s, sys: 1.05 s, total: 9.27 s
Wall time: 9.27 s


Item Based

In [8]:
'''calc item similarities based on features'''
%time IxI_sim_f=IxF_idf.dot(IxF_idf.T)
IxI_sim_f.setdiag(0)

CPU times: user 6.83 s, sys: 1.36 s, total: 8.19 s
Wall time: 8.19 s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [9]:
'''calc item similarities based on collaborative'''
%time IxI_sim_c=UxI_global1.T.dot(UxI_global1)
IxI_sim_c.setdiag(0)

CPU times: user 416 ms, sys: 64 ms, total: 480 ms
Wall time: 482 ms


/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [10]:
'''merge item similarities -  TUNE PARAMETER HERE'''
%time IxI_sim = IxI_sim_f + IxI_sim_c*0.001
del IxI_sim_f
del IxI_sim_c

CPU times: user 7.19 s, sys: 1.58 s, total: 8.77 s
Wall time: 8.77 s


In [11]:
'''take knn items -  TUNE PARAMETER HERE'''
IxI_sim_knn=sm.lil_matrix((n_items,n_items))
k=200
for i in tqdm(range(n_items)):    
    top_k_idx =IxI_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    IxI_sim_knn[i,top_k_idx]=IxI_sim[i,top_k_idx]
del IxI_sim 

In [12]:
'''calc predictions'''
%time UxI_pred_CI=UxI_global1.dot(IxI_sim_knn.T)
del IxI_sim_knn

CPU times: user 3.68 s, sys: 408 ms, total: 4.08 s
Wall time: 4.08 s


User Based

In [13]:
'''calc user similarities based on profiles'''
%time UxU_sim_p=UxF.dot(UxF.T) #numerators of cosine
%time cos=sm.csr_matrix(cosine_similarity(UxF)) #cosine
cos.data=1/cos.data
denominators=UxU_sim_p.multiply(cos) #get denominators of cosine
del cos
denominators.data+=1 #add shrinkage to cosine denominator
denominators.data=1/denominators.data
%time UxU_sim_p=UxU_sim_p.multiply(denominators) #calc shrinked cosine
del denominators
UxU_sim_p.setdiag(0)

CPU times: user 5.78 s, sys: 552 ms, total: 6.33 s
Wall time: 6.33 s
CPU times: user 14.8 s, sys: 3.06 s, total: 17.9 s
Wall time: 17.9 s
CPU times: user 2.44 s, sys: 524 ms, total: 2.97 s
Wall time: 2.97 s


/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [14]:
'''calc user similarities based on collaborative'''
%time UxU_sim_c=UxI_global1.dot(UxI_global1.T)
UxU_sim_c.setdiag(0)

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 83.2 ms


/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [15]:
'''merge user similarities -  TUNE PARAMETER HERE'''
%time UxU_sim = UxU_sim_p + UxU_sim_c*0.001
del UxU_sim_p
del UxU_sim_c

CPU times: user 1.88 s, sys: 532 ms, total: 2.42 s
Wall time: 2.41 s


In [16]:
'''take knn users -  TUNE PARAMETER HERE'''
UxU_sim_knn=sm.lil_matrix((n_users,n_users))
k=100
for i in tqdm(range(n_users)):    
    top_k_idx =UxU_sim.getrow(i).toarray()[0].argpartition(-k)[-k:]
    UxU_sim_knn[i,top_k_idx]=UxU_sim[i,top_k_idx]  
del UxU_sim

In [17]:
'''calc_predictions'''
%time UxI_pred_CU=UxU_sim_knn.dot(UxI_global1)

CPU times: user 2.99 s, sys: 224 ms, total: 3.22 s
Wall time: 3.22 s


In [19]:
del UxU_sim_knn
del UxI_basic
del UxI_global1
del UxF
del IxF_idf

Removal of already voted items, not mandatory

In [20]:
#create matrix for test users predictions
UxI_pred_CB_test=sm.lil_matrix((n_users,n_items))
UxI_pred_CI_test=sm.lil_matrix((n_users,n_items))
UxI_pred_CU_test=sm.lil_matrix((n_users,n_items))
UxI_pred_SVD_test=sm.lil_matrix((n_users,n_items))

In [21]:
#take only test users predictions
%time UxI_pred_CB_test[test_users,:]=UxI_pred_CB[test_users,:]
%time UxI_pred_CI_test[test_users,:]=UxI_pred_CI[test_users,:]
%time UxI_pred_CU_test[test_users,:]=UxI_pred_CU[test_users,:]
%time UxI_pred_SVD_test[test_users,:]=UxI_pred_SVD[test_users,:]

CPU times: user 21.2 s, sys: 3.47 s, total: 24.7 s
Wall time: 24.7 s
CPU times: user 11.5 s, sys: 404 ms, total: 11.9 s
Wall time: 11.9 s
CPU times: user 14 s, sys: 908 ms, total: 14.9 s
Wall time: 14.9 s
CPU times: user 21.8 s, sys: 4.4 s, total: 26.2 s
Wall time: 26.2 s


In [22]:
#remove items already voted
for user in tqdm(test_users):
    UxI_pred_CB_test[user,grouped_rates_dic[user]]=0
    UxI_pred_CI_test[user,grouped_rates_dic[user]]=0
    UxI_pred_CU_test[user,grouped_rates_dic[user]]=0
    UxI_pred_SVD_test[user,grouped_rates_dic[user]]=0

In [23]:
#convert to csr to improve future operations performances
%time UxI_pred_CB_test=UxI_pred_CB_test.tocsr()
%time UxI_pred_CI_test=UxI_pred_CI_test.tocsr()
%time UxI_pred_CU_test=UxI_pred_CU_test.tocsr()
%time UxI_pred_SVD_test=UxI_pred_SVD_test.tocsr()

CPU times: user 16.5 s, sys: 948 ms, total: 17.4 s
Wall time: 17.4 s
CPU times: user 816 ms, sys: 60 ms, total: 876 ms
Wall time: 875 ms
CPU times: user 3.36 s, sys: 72 ms, total: 3.43 s
Wall time: 3.43 s
CPU times: user 21.9 s, sys: 1.06 s, total: 23 s
Wall time: 23 s


In [24]:
del UxI_pred_CB
del UxI_pred_CI
del UxI_pred_CU
del UxI_pred_SVD

Algorithms adjusted with svd in order to remove possible even scores and conseguent random ordering

In [25]:
%time UxI_pred_CB_SVD = UxI_pred_CB_test + UxI_pred_SVD_test*0.000001

CPU times: user 1.36 s, sys: 376 ms, total: 1.73 s
Wall time: 1.73 s


In [26]:
%time UxI_pred_CI_SVD = UxI_pred_CI_test + UxI_pred_SVD_test*0.000001

CPU times: user 856 ms, sys: 332 ms, total: 1.19 s
Wall time: 1.19 s


In [27]:
%time UxI_pred_CU_SVD = UxI_pred_CU_test + UxI_pred_SVD_test*0.000001

CPU times: user 960 ms, sys: 372 ms, total: 1.33 s
Wall time: 1.33 s


In [28]:
del UxI_pred_CB_test
del UxI_pred_CI_test
del UxI_pred_CU_test
del UxI_pred_SVD_test

Testing print

In [ ]:
UxI_pred=UxI_pred_CU

In [ ]:
f = open('zzz_CU_modified.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))

for user in tqdm(test_users):
    top=[]

    user_predictions=user_predictions=(-UxI_pred.getrow(user).toarray()[0]).argsort()
    i=0
    while len(top)<=5:
        prediction=user_predictions[i]
        
        if UxI_pred[user,prediction]==0.0:
            j=0
            while len(top)<=5:
                prediction = item_ratings_mean[j]
                while prediction in grouped_rates_dic[user] or prediction in top:
                    j += 1
                    prediction = item_ratings_mean[j]
                top.append(prediction)
                j += 1
        
        
        
        if prediction in grouped_rates_dic[user]:
            i+=1
            continue
        
        i+=1
        top.append(prediction)
        
   
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

Real Print

In [29]:
'''TUNE PARAMETERS HERE'''
def bordaLikeAggr(rank1,rank2,rank3):    
    result=[0]*n_items
    rg=150
    for i in range(rg):
        item1=rank1.argmax()
        item2=rank2.argmax()
        item3=rank3.argmax()   
        
        if rank1[item1]>0.0:
            result[item1]+=(6/(i+1))
        rank1[item1]=-500

        if rank2[item2]>0.0:
            result[item2]+=(4/(i+1))
        rank2[item2]=-500
        
        if rank3[item3]>0.0:
            result[item3]+=(0.05/(i+1))
        rank2[item3]=-500
    
    return sm.csr_matrix(result)

In [30]:
f = open('submission_RangAgg_6-4-005.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in tqdm(test_users):
    top=[0,0,0,0,0]

    user_predictions=bordaLikeAggr(UxI_pred_CI_SVD.toarray()[0],UxI_pred_CB_SVD.getrow(user).toarray()[0],UxI_pred_CU_SVD.getrow(user).toarray()[0])
    iterator = 0
    for i in range(5):
        prediction = user_predictions.argmax()
        while prediction in grouped_rates_dic[user] and prediction != 0:
            user_predictions[0,prediction]=-9
            prediction=user_predictions.argmax()
        if prediction == 0:
            prediction = item_ratings_mean[iterator]
            while prediction in grouped_rates_dic[user] or prediction in top:
                iterator += 1
                prediction = item_ratings_mean[iterator]
            iterator += 1
        else:
            user_predictions[0,prediction]=-9
        top[i]=prediction    
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

KeyboardInterrupt: 

<15375x37143 sparse matrix of type '<class 'numpy.float64'>'
	with 141028612 stored elements in Compressed Sparse Row format>